In [30]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import talib as ta
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
 def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

In [ ]:
#theo tuần
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 50  # Initialize the investment fund

    def init(self):
        close = self.data.Close
        self.rsi = self.I(ta.RSI, close, timeperiod=14)
        self.boll_high, _, self.boll_low = self.I(ta.BBANDS, close, timeperiod=15, nbdevup=2, nbdevdn=2)
        self.day_of_week = self.I(
                            lambda x: x,
                            self.data.Close.s.index.dayofweek,
                            plot = False,
                            )

    def next(self):
        today = self.data.index[-1]
        
        if self.day_of_week[-1] == 1:
            self.fund += self.average_monthly_income_vnd * self.investment_percentage / 4
        
        # Check for buy signal based on Bollinger Bands and RSI
        if (self.data.Close[-1] <= self.boll_low[-1] and self.rsi[-1] <= 30):
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price
            shares_to_buy = (shares_to_buy // 100) * 100  # Round down to nearest 100 shares
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                
def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    #bt.plot(filename=f'{stock_symbol}')
    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['VCB', 'VPB', 'FPT', 'REE', "MSN", 'VNM']

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-19 23:18:35,326 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-08-19 23:18:36,037 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for VCB:
Total investment: 2641.3100579378106
Current Shares: 1000
Current Equity: 3473.792394655704
RoR: 31.517781648393434
--------------------------------------------------


2024-08-19 23:18:36,972 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for VPB:
Total investment: 2423.2013278104787
Current Shares: 4800
Current Equity: 3579.527235354574
RoR: 47.71893669226863
--------------------------------------------------


2024-08-19 23:18:37,802 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for FPT:
Total investment: 2386.382232289081
Current Shares: 1500
Current Equity: 5128.057553956834
RoR: 114.88835629813863
--------------------------------------------------
